In [ ]:
%matplotlib inline
from pprint import pprint

from matplotlib import pyplot as plt
import matplotlib

import simulate
from harvesting import N_60_RebalanceHarvesting, N_100_RebalanceHarvesting
import harvesting

import itertools
from decimal import Decimal
from montecarlo import conservative
import metrics
import mortality
import montecarlo
import lens
import pandas
import seaborn
import math
import types
from market import Returns_US_1871
import withdrawal

In [ ]:
seaborn.set(style="whitegrid")
seaborn.set_context('poster')

In [ ]:
def is_sequence(arg):
    return (not hasattr(arg, "strip") and
            hasattr(arg, "__iteritems__") or
            hasattr(arg, "__iter__"))

def listify(arg):
    if is_sequence(arg) and not isinstance(arg, dict):
        return arg
    return [arg,]

In [ ]:
def make_vpwfloor(floor):
    w = lambda p, h: withdrawal.VPWFloored(p, h, floor=floor)
    w.__name__ = f'VPWFloor_{floor}'
    return w

def run(series, lens, years=30, w1=make_vpwfloor(Decimal('.04')), w2=withdrawal.ConstantDollar):
    (r1, r2) = itertools.tee(series)
    portfolio = (600000, 400000)
    sim1 = simulate.withdrawals(r1, years=years, harvesting=N_60_RebalanceHarvesting, withdraw=w1, portfolio=portfolio)
    sim2 = simulate.withdrawals(r2, years=years, harvesting=N_60_RebalanceHarvesting, withdraw=w2, portfolio=portfolio)

    df = pandas.DataFrame(data={
        w1.__name__: listify(lens(sim1)),
        w2.__name__: listify(lens(sim2)),
    })
    return df

def run3(series, lens, years=30, w1=make_vpwfloor(Decimal('.04')), w2=withdrawal.ConstantDollar, w3=withdrawal.VPW):
    (r1, r2, r3) = itertools.tee(series, 3)
    portfolio = (600000, 400000)
    sim1 = simulate.withdrawals(r1, years=years, harvesting=N_60_RebalanceHarvesting, withdraw=w1, portfolio=portfolio)
    sim2 = simulate.withdrawals(r2, years=years, harvesting=N_60_RebalanceHarvesting, withdraw=w2, portfolio=portfolio)
    sim3 = simulate.withdrawals(r3, years=years, harvesting=N_60_RebalanceHarvesting, withdraw=w3, portfolio=portfolio)

    df = pandas.DataFrame(data={
        w1.__name__: listify(lens(sim1)),
        w2.__name__: listify(lens(sim2)),
        w3.__name__: listify(lens(sim3)),
    })    
    return df


def withdrawals(s):
    return [int(n.withdraw_r) for n in s]

def portfolio(s):
    return [int(n.portfolio_post.value_r) for n in s]

def cwd(s):
    return [float(n.withdraw_pct_cur) for n in s]

def pv_20(annual):
    portfolio_values = [n.portfolio_post.value_r for n in annual]
    s = pandas.Series(portfolio_values)
    return int(s.at[20])

def wd_20(annual):
    wd = [n.withdraw_pct_cur for n in annual]
    s = pandas.Series(wd)
    return float(s.at[20])

def get_rq(age, withdrawal_pct, stock_pct=60):
    # I can't figure out how to to joint life expectancy so I'll
    # just use female life expectancy for now :/
    life_expectancy = mortality.life_expectancy(None, age)
    mean = montecarlo.simba_mean[stock_pct]
    stddev = montecarlo.simba_stddev[stock_pct]
    
    return metrics.probability_of_ruin(mean, stddev, life_expectancy, float(withdrawal_pct))

def rq(annual):
    r_series = []
    for (i, age) in zip(annual, itertools.count(start=65)):
        r = get_rq(age, i.withdraw_n/i.portfolio_pre.value_n)
        r_series.append(r)
    return r_series

def sustained(annual, floor=Decimal('.04')):
    initial = annual[0].portfolio_pre.value_r
    # subtract $1 to deal with floating point weirdness (sometimes $40 turns into $39.9999)
    floor = floor * initial - 1

    df = pandas.DataFrame(annual)
    failed = df[df['withdraw_r'] < floor]
    s_y = len(failed)

    if s_y:
        return -s_y
    else:
        b_t = df.tail(1)['portfolio_post'].item().value_r
        b_y = b_t / df['withdraw_r'][0]
        return b_y


In [ ]:
US = Returns_US_1871()

In [ ]:
run(US.iter_from(1969), withdrawals).head()

In [ ]:
run(US.iter_from(1969), sustained).head()

In [ ]:
s1 = simulate.calc_lens(N_60_RebalanceHarvesting, make_vpwfloor(Decimal('.04')), 30, sustained)
s2 = simulate.calc_lens(N_60_RebalanceHarvesting, withdrawal.ConstantDollar, 30, sustained)

In [ ]:
s1[s1<0]

In [ ]:
s2[s2<0]

In [ ]:
df = run(US.iter_from(1969), portfolio)
plt.figure(figsize=(11,9))
g = seaborn.lineplot(data=df)
plt.title('Portfolio value for 1969 retirement')
seaborn.despine(left=True, bottom=True, offset=20)
g.axes.yaxis.set_major_formatter(
    matplotlib.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))

In [ ]:
s1 = simulate.calc_lens(N_60_RebalanceHarvesting, make_vpwfloor(Decimal('.034')), 30, pv_20)
plt.figure(figsize=(11,9))
g = seaborn.lineplot(data=s1)
plt.title('Portfolio value after 20 years with 3.4% floor')
seaborn.despine(left=True, bottom=True, offset=20)
g.axes.yaxis.set_major_formatter(
    matplotlib.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))

In [ ]:
s1 = simulate.calc_lens(N_60_RebalanceHarvesting, make_vpwfloor(Decimal('.034')), 30, wd_20)
plt.figure(figsize=(11,9))
g = seaborn.lineplot(data=s1)
plt.title('Current withdrawal rate in Year 20 with 3.4% floor')
seaborn.despine(left=True, bottom=True, offset=20)

In [ ]:
a = simulate.withdrawals(US.iter_from(1969),
                         years=30,
                         harvesting=N_60_RebalanceHarvesting,
                         withdraw=make_vpwfloor(Decimal('.04')),
                         portfolio=(600_000, 400_000))

In [ ]:
sf = []
for i in range(10):
    floor = Decimal('4')
    floor = floor - Decimal(i)/10
    floor = floor / 100
    s1 = simulate.calc_lens(N_60_RebalanceHarvesting, make_vpwfloor(floor), 30, lambda x: sustained(x, floor=floor))
    sf.append((floor, -s1[s1<0].sum()))

In [ ]:
s = pandas.Series(data=[x[1] for x in sf], index=[x[0] for x in sf])
plt.figure(figsize=(11,9))
plt.title('Years of shortfall')
seaborn.despine(left=True, bottom=True, offset=20)
s.plot()
s

In [ ]:
df = run3(US.iter_from(1969), withdrawals, w1=make_vpwfloor(Decimal('0.034')))
plt.figure(figsize=(11,9))
g = seaborn.lineplot(data=df)
plt.title('Withdrawals for 1969 retirement')
seaborn.despine(left=True, bottom=True, offset=20)
g.axes.yaxis.set_major_formatter(
    matplotlib.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))

In [ ]:
s1 = simulate.calc_lens(N_60_RebalanceHarvesting, make_vpwfloor(Decimal('.03')), 30, wd_20)
plt.figure(figsize=(11,9))
g = seaborn.lineplot(data=s1)
plt.title('Current withdrawal rate in Year 20 with 3.4% floor')
seaborn.despine(left=True, bottom=True, offset=20)

In [ ]:
df = run(US.iter_from(1969), cwd, w1=make_vpwfloor(Decimal('0.034')), w2=withdrawal.VPW)
plt.figure(figsize=(11,9))
g = seaborn.lineplot(data=df)
plt.title('Current withdrawal percentage for 1969 retirement')
seaborn.despine(left=True, bottom=True, offset=20)

In [ ]:
df = run(US.iter_from(1969), cwd, w1=make_vpwfloor(Decimal('0.031')), w2=withdrawal.VPW)
plt.figure(figsize=(11,9))
g = seaborn.lineplot(data=df)
plt.title('Current withdrawal percentage for 1969 retirement')
seaborn.despine(left=True, bottom=True, offset=20)

In [ ]:
df = run(US.iter_from(1969), rq, w1=make_vpwfloor(Decimal('0.034')), w2=withdrawal.VPW)
plt.figure(figsize=(11,9))
g = seaborn.lineplot(data=df)
plt.title('Risk Quotient for 1969 retirement')
seaborn.despine(left=True, bottom=True, offset=20)

In [ ]:
sf = []
for i in range(15):
    floor = Decimal('3.4')
    floor = floor - Decimal(i)/10
    floor = floor / 100
    df = run(US.iter_from(1969), rq, w1=make_vpwfloor(floor), w2=withdrawal.VPW)
    n = df.iloc[0:20, 0].max()
    sf.append((floor, n))

In [ ]:
s = pandas.Series(data=[x[1] for x in sf], index=[x[0] for x in sf])
plt.figure(figsize=(11,9))
plt.title('Max risk quotient')
seaborn.despine(left=True, bottom=True, offset=20)
s.plot()
s

In [ ]:
df = run(US.iter_from(1969), rq, w1=make_vpwfloor(Decimal('0.030')), w2=withdrawal.VPW)
plt.figure(figsize=(11,9))
g = seaborn.lineplot(data=df)
plt.title('Risk Quotient for 1969 retirement')
seaborn.despine(left=True, bottom=True, offset=20)
df.iloc[0:20, 0].max()

In [ ]:
for i in range(10):
    floor = Decimal(3)
    floor = floor - Decimal(i)/10
    floor = floor / 100
    df = run(US.iter_from(1969), withdrawals, w1=make_vpwfloor(floor), w2=withdrawal.VPW)
    plt.figure(figsize=(11,9))
    g = seaborn.lineplot(data=df)
    plt.title(f'1969 - Withdrawals with floor {floor}')
    seaborn.despine(left=True, bottom=True, offset=20)
    g.axes.yaxis.set_major_formatter(
        matplotlib.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))

In [ ]:
df = run(US.iter_from(1969), withdrawals, w1=make_vpwfloor(Decimal('0.030')), w2=withdrawal.VPW)
#new_df = df['VPWFloor_0.028'] - df['VPW']
new_df = df['VPWFloor_0.030'] / df['VPW']
new_df[new_df > 1]

In [ ]:
df